[Link na zadatke](http://www.fer.unizg.hr/_download/repository/vjezba_2%5B4%5D.pdf)

## Funkcije cilja

In [ ]:
import numpy as np

def f1(x):
    x1, x2 = x
    return 100 * (x2 - x1 ** 2) ** 2 + (1 - x1) ** 2
f1_x0 = (-1.9, 2)
f1_xmin = (1, 1)
f1_min = 0

def f2(x):
    x1, x2 = x
    return (x1 - 4) ** 2 + 4 * (x2 - 2) ** 2
f2_x0 = (0.1, 0.3)
f2_xmin = (4, 2)
f2_min = 0

def f3(x):
    return np.sum(np.square(np.arange(1, x.shape[0] + 1) - x))
f3_x0 = (0, 0)
f3_min = 0

def f4(x):
    x1, x2 = x
    return abs((x1 - x2) * (x1 + x2)) + math.sqrt(x1 ** 2 + x2 ** 2)
f4_x0 = (5.1, 1.1)
f4_min = 0

def f6(x):
    x = np.sum(np.square(x))
    return 0.5 + (math.sin(math.sqrt(x)) ** 2 - 0.5) / (1 + 0.001 * x) ** 2
f6_xmin = 0
f6_min = 0

### [Golden section](http://www.fer.unizg.hr/_download/repository/zlatni_rez.txt)

In [ ]:
def zlatni_rez(a, b, e, k, f):
    hi = b - k * (b - a)
    lo = a + k * (b - a)
    
    f_hi = f(hi)
    f_lo = f(lo)
    
    while (b - a) > e:
        if f_hi < f_lo:
            b = lo
            lo = hi
            hi = b - k * (b - a)
            f_lo = f_hi
            f_hi = f(hi)
        else:
            a = hi
            hi = lo
            lo = a + k * (b - a)
            f_hi = f_lo
            f_lo = f(lo)
    
    return (a + b) / 2

### [Unimodal](http://www.fer.unizg.hr/_download/repository/unimodalni.txt)

In [ ]:
def unimodalni(h, pt):
    l, r = pt - h, pt + h
    m = pt
    step = 1
    
    fm = f(pt)
    fl = f(l)
    fr = f(r)
    
    if fm < fr and fm < fl:
        return l, r
    else if fm > fr:
        while True:
            l, m, fm = m, r, fr
            r = pt + h * (step * 2)
            fr = f(r)
            step *= 2
            if fm <= fr:
                return l, r
    else:
        while True:
            r, m, fm = m, l, fl
            l = pt - h * (step * 2)
            fl = f(l)
            step *= 2
            if fm <= fl:
                return l, r
    

### [Simplex](http://www.fer.unizg.hr/_download/repository/simplex.html)

In [ ]:
def simplex_nelder_mead(
    x0, alpha=1, beta=0.5, gamma=2, sigma=0.5 epsilon=1e-6):
    
    x = simplex_pts(x0)
    reflect = lambda *none: x0 + alpha * (x0 - x[-1])
    expand = lambda xr: x0 + gamma * (xr - x0)
    
    while True:
        xc = np.mean(x)
        xr = reflect()
        
        if f(xr) < f(x[l]):
            xe = expand(xr)
            if f(xe) < f(x[l]):
                x[h] = xe
            else:
                x[h] = xr
        else:
            fxr = f(xr)
            if all(fxr > f(xj) for j, xj in enumerate(x) if j != h):
                if fxr < f(x[h]):
                    x[h] = xr
                else:
                    pass # treba pomaknut sve tocke prema X[l] ??
            else:
                x[h] = xr
        
        if stopping_condition(x):
            break
            
    return x

### [Hooke-Jeeves](http://www.fer.unizg.hr/_download/repository/hj.html)

In [ ]:
def hooke_jeeves():
    